# Set tensorflow to use only second GPU only

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# *Setting S3 credential environment (Storing artifact)*

In [ ]:
import os

os.environ["AWS_ACCESS_KEY_ID"] = "admin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "sample_key"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://192.168.1.12:9000"

# Import tensorflow module

In [2]:
import tensorflow.keras.layers as layers
import tensorflow.keras as tfk
from tensorflow.keras.datasets.mnist import load_data


2022-06-02 10:27:26.276928: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# ***MLFlow Init***

In [3]:
import mlflow

mlflow.set_tracking_uri("http://192.168.1.12:5000")
mlflow.set_experiment(experiment_name="mnist-classification")
mlflow.tensorflow.autolog()

# Load MNIST dataset

In [4]:
(X_train, y_train), (X_test, y_test) = load_data()

X_train = (X_train / 255.0).reshape(-1, 28, 28, 1)
X_test = (X_test / 255.0).reshape(-1, 28, 28, 1)


# Create Model

In [5]:
model = tfk.Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(1, 3, (1, 1), padding='same'),
    layers.BatchNormalization(),

    layers.Conv2D(32, 3, (1, 1), padding='same'),
    layers.LeakyReLU(),
    layers.BatchNormalization(),
    layers.MaxPool2D(2),

    layers.Conv2D(64, 3, (1, 1), padding='same'),
    layers.LeakyReLU(),
    layers.BatchNormalization(),

    layers.Flatten(),
    
    layers.Dense(10, activation='softmax')
])


2022-06-02 10:27:28.826433: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-02 10:27:29.278416: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11319 MB memory:  -> device: 0, name: Tesla P100-PCIE-12GB, pci bus id: 0000:65:00.0, compute capability: 6.0


In [6]:
model.compile(
    optimizer=tfk.optimizers.Adam(learning_rate=0.0001),
    loss=tfk.losses.SparseCategoricalCrossentropy(),
    metrics=['acc']
)


In [7]:
print("Input shape", X_train.shape, X_test.shape)

Input shape (60000, 28, 28, 1) (10000, 28, 28, 1)


# Training Model

In [9]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10


2022-06-02 10:27:31.840150: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


1875/1875 [==============================] - 14s 7ms/step - loss: 0.2083 - acc: 0.9358 - val_loss: 0.0918 - val_acc: 0.9717
Epoch 2/10
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0801 - acc: 0.9758 - val_loss: 0.0665 - val_acc: 0.9798
Epoch 3/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0552 - acc: 0.9826 - val_loss: 0.0748 - val_acc: 0.9776
Epoch 4/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0403 - acc: 0.9871 - val_loss: 0.0784 - val_acc: 0.9765
Epoch 5/10
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0316 - acc: 0.9899 - val_loss: 0.0675 - val_acc: 0.9800
Epoch 6/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0223 - acc: 0.9926 - val_loss: 0.0632 - val_acc: 0.9820
Epoch 7/10
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0171 - acc: 0.9945 - val_loss: 0.0642 - val_acc: 0.9823
Epoch 8/10
1875/1875 [==============================] - 12s 7ms/st

INFO:tensorflow:Assets written to: /tmp/tmpfb2hue3z/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpfb2hue3z/model/data/model/assets
2022/06/02 10:29:39 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpfb2hue3z/model, flavor: keras), fall back to return ['tensorflow==2.9.1', 'keras==2.9.0']. Set logging level to DEBUG to see the full traceback.
2022/06/02 10:29:39 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/jupyter/anaconda3/envs/tf-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
